In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import warnings
import pandas as pd

warnings.filterwarnings("ignore")
pd.options.display.float_format = "{:,.2f}".format

In [5]:
import dgl_utils as du
from stgnn import STGCN_WAVE, evaluate_model

In [6]:
data = du.get_data()
train_indices, val_indices, test_indices = du.split(data, train_ratio=0.9, val_ratio=0.1)
A = du.generate_adjacency_matrix(data.loc[train_indices, "production"], threshold=0.6)
G = du.create_graph(A)

x, y = du.data_transform(data, n_window=48, n_ahead=1)
train_iter, val_iter = du.prepare_torch_data(x, y, train_indices, val_indices, n_window=48, batch_size=32)

1487 61.958333333333336
5405
5405


In [ ]:
data.head()

# # scaling'i çözmek gerekecek
# # pd.DataFrame(scaler.fit_transform(data.values)).mean(axis=0)

# from sklearn.preprocessing import StandardScaler  
# scaler = StandardScaler()

# data.loc[train_indices] = scaler.fit_transform(data.loc[train_indices])

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from stgnn import STGCN_WAVE, evaluate_model

In [ ]:
blocks = [1, 16, 32, 64, 32, 128]
lr = 0.001
num_nodes = 97

loss = nn.MSELoss()
device=torch.device("cpu")
G = G.to(device)
model = STGCN_WAVE(blocks, n_window, num_nodes, G).to(device)

optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

save_path = "stgcnwavemodel.pt"
epochs = 5

from time import time
min_val_loss = np.inf

for epoch in range(1, epochs + 1):
    l_sum, n = 0.0, 0
    model.train()
    t_ = time()
    for x, y in train_iter:
        y_pred = model(x).view(len(x), -1)
        l = loss(y_pred, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        l_sum += l.item() * y.shape[0]
        n += y.shape[0]
    scheduler.step()
    val_loss = evaluate_model(model, loss, val_iter)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), save_path)
    print("epoch", epoch, ", time:", round(time() - t_, 2),  ", train loss:", l_sum / n, ", validation loss:", val_loss)

    

In [ ]:
du.convert_x_tensor_to_df(x, 0, -1)

In [ ]:
du.convert_y_tensor_to_df(y, 0)